In [1]:
import pandas as pd
import numpy as np
import random
from pyomo.environ import *
from itertools import combinations

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
roud = 'C:/Jupyter/TESIS/instancias/real/testInstance_Amtrak.csv'
start_estation = 'NYP'
jerar_comer_class = {'Y': '1-1','YA': '1-2','YB': '1-3','YC': '1-4','YD': '1-5','YE': '1-6','YF': '1-7','YG': '1-8','JY': '2-1','JA': '2-2','JB': '2-3','JC': '2-4','JD': '2-5','JE': '2-6','JF': '2-7','JG': '2-8'}

In [3]:
def clean_data(roud, start_estation, jerar_comer_class):
    # read data
    data = pd.read_csv(roud, delimiter=';')
    data.drop(columns=['Unnamed: 0'], inplace=True)

    # fiend parameters
    origin = data['Origin'].unique().tolist()
    destin = data['Destination'].unique().tolist()
    periodos = sorted(data['DBD'].unique().tolist(), reverse=True)

    stations = destin + [i for i in origin if i not in destin]

    anonymo_stat = {i: data[data['Market'] == start_estation + '-' + i]['NbLegs'].unique()[0] + 1 if i != start_estation else 1 for i in stations }
    data.replace(anonymo_stat, inplace=True)
    origin_cor = sorted(data['Origin'].unique().tolist())
    destin_cor = sorted(data['Destination'].unique().tolist())

    data['Market'] = data['Origin'].astype('str') +'-'+ data['Destination'].astype('str')
    oridest = data['Market'].unique().tolist()
    oridest = sorted([(int(i.split('-')[0]), int(i.split('-')[1])) for i in oridest])

    data['CommercialClass'].replace(jerar_comer_class, inplace=True)

    data[['vagon','clase']] = data['CommercialClass'].str.split('-', expand=True).astype('int')
    vagones = data['vagon'].unique().tolist()
    clases = {v: sorted(data[data['vagon']==v]['clase'].unique().tolist()) for v in vagones}

    demanda = pd.pivot_table(data, values=['Bookings'], index=['Origin','Destination','vagon','clase', 'DBD'], aggfunc={'Bookings':'sum'})
    demanda.reset_index(inplace=True)

    preco = pd.pivot_table(data, values=['Revenue'], index=['Origin','Destination','vagon','clase'], aggfunc={'Revenue':'max'})
    preco.reset_index(inplace=True)

    for i,j in oridest:
        for v in vagones:
            for k in clases[v]:
                    if preco.loc[(preco.Origin == i) & (preco.Destination == j) & (preco.vagon == v) & (preco.clase == k)]['Revenue'].count() == 0:
                        a = {'Origin':i,'Destination':j,'vagon':v,'clase':k,'Revenue':0}
                        preco.loc[len(preco)] = a

    for i,j in oridest:
        for v in vagones:
            for k in clases[v]:
                    for t in periodos:
                        if demanda.loc[(demanda.Origin == i) & (demanda.Destination == j) & (demanda.vagon == v) & (demanda.clase == k) & (demanda.DBD == t)]['Bookings'].count() == 0:
                            a = {'Origin':i,'Destination':j,'vagon':v,'clase':k, 'DBD':t,'Bookings':0}
                            demanda.loc[len(demanda)] = a

    preco = preco.sort_values(by=['Origin', 'Destination', 'vagon', 'Revenue'], ascending=[True, True, True, False])

    dem_cor = demanda.set_index(['Origin', 'Destination', 'vagon', 'clase', 'DBD'])['Bookings'].to_dict()
    preco_cor = preco.set_index(['Origin', 'Destination', 'vagon', 'clase'])['Revenue'].to_dict()

    return origin_cor, destin_cor, oridest, vagones, periodos, stations, clases, preco_cor, dem_cor


In [4]:

origin1, destin, oridest, vagones, periodos, stations, clases, preco, dem = clean_data(roud, start_estation, jerar_comer_class)
origin2 = [0] + origin1
n = len(stations)

In [5]:
cap = 300

In [ ]:
# instanciar o modelo
model = AbstractModel()

# Escalares
model.n = Param(initialize = n)
model.Q = Param(initialize = cap)

# conjuntos
model.I1 = Set(initialize = origin1)
model.I2 = Set(initialize = origin2)
model.J = Set(initialize = destin)
model.OD = Set(dimen=2, initialize = oridest)
model.V = Set(initialize = vagones)
model.T = Set(initialize = periodos)
# model.K = Set(initialize = clases)

def xxx(model):
    return ((v,k) for v in clases.keys() for k in clases[v])
model.VKI = Set(dimen=2,initialize=xxx)

model.VK = Set(model.V, initialize=clases)
    
#parâmetros indexados
model.P = Param(model.OD, model.VKI, initialize = preco)
model.d = Param(model.OD, model.VKI, model.T, initialize = dem)

# variáveis de decisão
model.X = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
model.Y = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
model.A = Var(model.I2, within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum(model.P[i,j,v,k]*(model.X[i,j,v,k,t]) for i,j in model.OD for v,k in model.VKI for t in model.T)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def cap1(model, i):
    return model.A[i] ==  model.A[i-1] - sum((model.X[i-1,j,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T if j>=i) + sum((model.X[j,i,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T if j<i)
model.ConstrainCap1 = Constraint(model.I1, rule = cap1)

def cap2(model, i,_):
    if i < model.n:
        return sum((model.X[i,j,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T) <= model.A[i]
    else:
        return Constraint.Skip
model.ConstrainCap2 = Constraint(model.OD, rule = cap2)

def authorization(model, i, j, v, k, t):
    if i < j:
        return model.Y[i,j,v,k,t] >= model.X[i,j,v,k,t]
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.OD, model.VKI, model.T, rule = authorization)

def classe(model, i, j, v, k, t):
    lista = list(model.VK[v])
    if k != lista[-1]:
        pos_k = lista.index(k)
        return model.Y[i,j,v,k,t] >= model.Y[i,j,v,lista[pos_k+1],t]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.OD, model.VKI, model.T, rule = classe)

def assignment(model, i, j, v, k, t):
    if i < j:
        return model.X[i,j,v,k,t] <= model.d[i,j,v,k,t]
    else:
        return Constraint.Skip
model.ConstrainAssignment = Constraint(model.OD, model.VKI, model.T,rule = assignment)

def assignment0(model, _, j, v, k, t):
    return model.X[0,j,v,k, t] == 0
model.ConstrainAssignment0 = Constraint(model.OD, model.VKI, model.T, rule = assignment0)

def authorization0(model, _, j, v, k, t):
    return model.Y[0,j,v,k,t] == 0
model.ConstrainAuthorization0 = Constraint(model.OD, model.VKI, model.T, rule = authorization0)

def cap0(model):
    return model.A[0] == model.Q
model.ConstrainCap0 = Constraint(rule = cap0)

instance = model.create_instance()
# instance.pprint()

opt = SolverFactory('cplex')
results = opt.solve(instance, tee=True)
# print(results)

In [ ]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i,j in instance.OD:
    for v,k in instance.VKI:
        for t in instance.T:
            # if value(instance.Y[i,j,v,k,t])!=0 or value(instance.X[i,j,v,k,t])!=0:
                # print(i,j,v,k,t, ' - ',value(instance.Y[i,j,v,k,t]), ' - ', value(instance.X[i,j,v,k,t]) )
            lista.append([str(i) + '-' + str(j),v, k, t, value(instance.P[i,j,v,k]), value(instance.d[i,j,v,k,t]) , value(instance.X[i,j,v,k,t]), value(instance.Y[i,j,v,k,t]) ])

a = pd.DataFrame(lista, columns=['o-d','vagon','classe','Periodo','preco','demanda','Assignments','Authorizations'])
a.to_excel('prueba.xlsx')

In [47]:
# list(instance.)
value(instance.n)

11

In [44]:
# dem = {(i,j,v,k,t): np.random.randint(1,100) for i,j in oridest for v in vagones for k in clases for t in periodos }

# preco = {}
# for i,j in oridest:
#     for v in vagones:
#         aleat = sorted(random.sample(range(50, 2000 + 1), len(clases)), reverse=True)
#         for k,valor in enumerate(aleat):
#             preco[(i,j,v,clases[k])] = valor


# oridest = [(1,2),(1,3),(1,4),(1,5)]
# origin1 = [1]
# origin2 = [0,1]
# destin = [2,3,4,5]
# vagones = [1,2]
# periodos = [0,1,5,10]

# cap = 200 
# n = 5 
# clases = {1:[1,5,7],2:[1,5,7]}
# # clases = [1,5,7]

# corrigir instancias
# dem = {(i,j,v,k,t): dem[(i,j,v,k,t)] if (i,j,v,k,t) in dem else 0  for i,j in oridest for v in vagones for k in clases_ord[v] for t in periodos}
# preco = {(i,j,v,k): preco[(i,j,v,k)] if (i,j,v,k) in preco else 0  for i,j in oridest for v in vagones for k in clases_ord[v] }